# Note
This notebooks uses the file generated by the notebook `process_dates_screen_storage.ipynb`.

### Dealing with imports...

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
# Import Area Under the Receiver Operating Characteristic Curve metric to evaluate results
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import Pipeline

In [2]:
from sklearn import preprocessing
from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction import FeatureHasher
from sklearn.preprocessing import LabelBinarizer

### Loading data...
The files loaded are generated by the notebook `process_dates_screen_starge.ipynb`. 

In [3]:
train_df = pd.read_csv('/home/miki_mustard/Desktop/Facultad/Datos/TP2/train_df_processed_screenResol_storage_dates.csv')

to_predict = pd.read_csv('/home/miki_mustard/Desktop/Facultad/Datos/TP2/to_predict_processed_screenResol_storage_dates.csv')

In [ ]:
train_df.shape

In [ ]:
to_predict.shape

In [ ]:
train_df.shape

In [ ]:
to_predict.shape

***
## Note on preprocessing
All preprocessing which can be done in just one way, i.e. it doesn't need hyper parameter adjustment, will be done outside pipelines and then stored to a new file, so there will be no need to execute the same code every time we open this notebook again.

Good pipeline resources: 
* http://zacstewart.com/2014/08/05/pipelines-of-featureunions-of-pipelines.html
* https://www.kaggle.com/baghern/a-deep-dive-into-sklearn-pipelines
* https://www.kaggle.com/metadist/work-like-a-pro-with-pipelines-and-feature-unions
* https://www.kaggle.com/sermakarevich/sklearn-pipelines-tutorial
* http://zacstewart.com/2014/08/05/pipelines-of-featureunions-of-pipelines.html
***

### Build some custom transformers

# Feature pipeline creation

### Data split

In [4]:
test_size = 0.33
# define a seed, so same experiments output same results every time and experiments between them become comparable
seed = 12

# realizo train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_df.loc[:, train_df.columns != 'label'], 
                                                    train_df.label, 
                                                    test_size=test_size, 
                                                    random_state=seed)

## Pipelines

In [5]:
from sklearn.feature_extraction.text import HashingVectorizer

from xgboost import XGBRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.compose import ColumnTransformer

from sklearn.impute import SimpleImputer

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.feature_extraction.text import TfidfTransformer

from sklearn.feature_extraction.text import CountVectorizer

from sklearn.decomposition import TruncatedSVD

In [ ]:
# hv = FeatureHasher(input_type='string')
# hv.fit_transform(X_train.event.fillna(""))

In [6]:
train_df.select_dtypes('object').columns

Index(['person', 'url', 'model', 'color', 'skus', 'search_term', 'city',
       'region', 'country'],
      dtype='object')

In [ ]:
# limited_categorical_transformer = Pipeline([
#     ("imputer",SimpleImputer(strategy='most_frequent')),
#     ("one_hot",OneHotEncoder(handle_unknown='ignore'))
# ])

In [ ]:
# large_categorical_transformer = Pipeline([
#     ("imputer",SimpleImputer(strategy='most_frequent')),
#     ("hashing_trci",FeatureHasher(input_type='string'))
# ])

In [ ]:
# tf_idf = Pipeline([
#     ("imputer",SimpleImputer(strategy='constant',fill_value="")),
#     ("tf_idf",TfidfVectorizer()),
#     ('best', TruncatedSVD())
# ])

In [ ]:
# from sklearn.compose import make_column_transformer

In [ ]:
# ct = make_column_transformer(
#     ("large_cat",large_categorical_transformer,["person","skus","city","region","country"]),
#     (TfidfVectorizer(),['model','color'])
#     ("tf_idf2",tf_idf,'color'),
#     ("tf_idf_reduced",TfidfVectorizer(),"search_term"),
#     ("ulr_tf",TfidfVectorizer(),"url")

#                        ,n_jobs=-1
# )

In [ ]:
# X_train.

In [ ]:
# X_train.color.fillna("", inplace=True)
# X_train.model.fillna("", inplace=True)

In [ ]:
# my_tf = TfidfVectorizer()
# my_tf.fit(X_train.color)

In [ ]:
preds_grid_search = random_search.predict(X_test)

In [7]:
feature_processing = Pipeline([
#     ('preproc', ct),
    ('xg_reg', XGBRegressor(objective ='binary:logistic', learning_rate=0.1, booster='dart'))
])

In [8]:
feature_processing.fit(X_train.select_dtypes(exclude='object'), y_train)

Pipeline(memory=None,
     steps=[('xg_reg', XGBRegressor(base_score=0.5, booster='dart', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1))])

In [9]:
preds_reg = feature_processing.predict(X_test.select_dtypes(exclude='object'))

In [ ]:
preds = feature_processing.predict_proba(X_test.select_dtypes(exclude='object'))[:,1]

In [10]:
# check shape of predictions
if (preds_reg.shape == y_test.shape):
    print('shapes OK')

shapes OK


In [11]:
# prev: 0.8886417505271881
#       0.7675200441464977
#       0.8501359056263651
#       0.9994019683472557    logistic_regression con preprocesamiento de browswer y os.
#       0.8501359056263651    idem pero con xgb
#       0.8503577934604342    xgb con:
#                                     ct = ColumnTransformer([
#                                         ("lim_cat",limited_categorical_transformer,["event","condition","staticpage",
#                                                                                     "campaign_source","search_engine",
#                                                                                     "channel","new_vs_returning","device_type",
#                                                                                     "operating_system_version","browser_version"]),
#                                         ("large_cat",large_categorical_transformer,["person","url","skus","city","region","country"]),
#                                         ("tf_idf",TfidfVectorizer(),"model"),
#                                         ("tf_idf2",TfidfVectorizer(),"color"),
#                                         ("tf_idf_reduced",TfidfVectorizer(),"search_term"),
#                                         ("passthrough",'passthrough',["storage","sku","year","month_sin","month_cos","day_sin","day_cos","weekday_sin",
#                                                                       "weekday_cos","hour_sin","hour_cos","screen_width","screen_height"])
#                                     ],n_jobs=-1)
roc_auc_score(y_test,preds_reg)

0.8537931349192909

In [ ]:
# feature_processing.named_steps['preproc'].named_transformers_['tf_idf'].idf_

## Export area

In [ ]:
preds_reg = feature_processing.predict(to_predict.select_dtypes(exclude='object'))

In [ ]:
#preds_posta = feature_processing.predict_proba(to_predict.select_dtypes(exclude='object'))[:,1]

In [ ]:
preds_posta

In [ ]:
to_publish = pd.DataFrame()

In [ ]:
to_publish['person'] = to_predict.person

In [ ]:
to_publish['label'] = preds_reg

In [ ]:
to_publish.shape

In [ ]:
to_publish.groupby('person', as_index=False).mean().to_csv('prediccion_entrega6.csv', index=False)